# Teste da API Comtrade

Info: https://comtradedeveloper.un.org

Specs: https://comtradedeveloper.un.org/api-details#api=comtrade-v1&operation=get-get



In [25]:
# %pip install requests

Note: you may need to restart the kernel to use updated packages.


In [26]:
# %pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [27]:
# %pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


Ler os códigos HS

Obter em https://github.com/datasets/harmonized-system/blob/master/data/harmonized-system.csv

Copiar para directoria `support`


In [62]:
import pandas as pd

hs_codes_df = pd.read_csv('downloads/support/harmonized-system.csv')
hs_codes_map = dict(zip(hs_codes_df.hscode, hs_codes_df.description))

Fazer um dicionário para os códigos de nível 2

In [63]:
hs_codes_l2 = hs_codes_df[hs_codes_df.level == 2]
hs_codes_l2[['hscode','description']]
hs_l2_map = dict(zip(hs_codes_l2.hscode, hs_codes_l2.description))



Aqui colocar os parâmetros gerais que não mudam.

In [30]:
baseUrl = "https://comtradeapi.un.org/public/v1"

m49_codes_cn_plp ={
    "024":"Angola",
    "076":"Brazil",
    "132":"Cabo Verde",
    "156":"China",
    "344":"China, Hong Kong",
    "446":"China, Macau",
    "226":"Guiné Equatorial",
    "624":"Guiné-Bissau",
    "508":"Moçambique",
    "620":"Portugal",
    "678":"São Tomé e Príncipe",
    "626":"Timor-Leste"
}
m49_china = "156"
m49_hong_kong = "344"
m49_macau = "446"
m49_plp = set(m49_codes_cn_plp.keys())-{m49_china,m49_hong_kong,m49_macau}
m49_plp_list = ",".join(m49_plp)

Aqui os parâmetros de cada chamada à API.

In [68]:
typeCode = "C" # C for commodities, S for Services
freqCode = "A" # A for annual and M for monthly
clCode = "HS" #Trade classifications Available values : HS, SS, B4, B5, EB, EB10, EB02, EBSDMX
reporterCode = m49_china # China, HK 344, Macau 446, see https://unstats.un.org/unsd/methodology/m49/
partnerCode = m49_plp_list
period="2021"
cmdCode="AG2" # TOTAL, AG2 (HS level 2)
flowCode = "M" # M=import, X=export,outros: RX, RM, MIP, XIP, MOP, XOP, MIF, XIF, DX, FM


In [69]:
import json
import requests
import pandas as pd

requestUrl=f"{baseUrl}/preview/{typeCode}/{freqCode}/{clCode}"
resp = requests.get(requestUrl,
        {
        'reporterCode':m49_china,
        'period':period,
        'partnerCode':m49_plp_list,
        'cmdCode':cmdCode,
        'flowCode':flowCode
        },
        timeout=10)
results = json.loads(resp.content)['data']
df = pd.DataFrame(results)
# convert result country codes to three-digit codes
df.reporterCode = df.reporterCode.astype('str').str.zfill(3)
df.partnerCode = df.partnerCode.astype('str').str.zfill(3)
df.reporterDesc = df.reporterCode.map(m49_codes_cn_plp)
df.partnerDesc = df.partnerCode.map(m49_codes_cn_plp)
# Map hscodes
df.cmdDesc = df.cmdCode.map(hs_codes_map)
df['primaryValueFormated'] = df.primaryValue.map('{:,}'.format)
# print(df.columns.values)

In [70]:
cols = ['typeCode','freqCode','reporterDesc','partnerDesc','refYear','cmdCode','cmdDesc','flowCode','primaryValueFormated','primaryValue']

pco = df[cols].sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] < 6]
pco_top5

,typeCode,freqCode,reporterDesc,partnerDesc,refYear,cmdCode,cmdDesc,flowCode,primaryValueFormated,primaryValue,rank
3,C,A,China,Angola,2021,27,"Mineral fuels, mineral oils and products of th...",M,"20,823,873,963.0",2.082387e+10,1.0
1,C,A,China,Angola,2021,25,"Salt; sulphur; earths, stone; plastering mater...",M,"30,420,529.0",3.042053e+07,2.0
2,C,A,China,Angola,2021,26,"Ores, slag and ash",M,"16,912,288.0",1.691229e+07,3.0
13,C,A,China,Angola,2021,74,Copper and articles thereof,M,"14,969,851.0",1.496985e+07,4.0
14,C,A,China,Angola,2021,76,Aluminium and articles thereof,M,"10,401,908.0",1.040191e+07,5.0
44,C,A,China,Brazil,2021,26,"Ores, slag and ash",M,"40,308,600,464.0",4.030860e+10,1.0
31,C,A,China,Brazil,2021,12,Oil seeds and oleaginous fruits; miscellaneous...,M,"33,120,876,740.0",3.312088e+10,2.0
45,C,A,China,Brazil,2021,27,"Mineral fuels, mineral oils and products of th...",M,"15,428,749,560.0",1.542875e+10,3.0
21,C,A,China,Brazil,2021,02,Meat and edible meat offal,M,"7,799,680,540.0",7.799681e+09,4.0
63,C,A,China,Brazil,2021,47,Pulp of wood or other fibrous cellulosic mater...,M,"4,344,336,176.0",4.344336e+09,5.0


In [71]:
df[cols].to_excel(f"./downloads/dados_comtrade_{period}_{cmdCode}.xlsx")
pco.to_excel(f"./downloads/paises_cmd_ordenados_{period}.xlsx")
pco_top5.to_excel(f"./downloads/paises_cmd_top5_{period}.xlsx")

Obter a lista dos códigos HS de nível 2 envolvidos

In [77]:
countries = pco_top5.partnerDesc.unique()
countries

array(['Angola', 'Brazil', 'Cabo Verde', 'Guiné Equatorial',
       'Guiné-Bissau', 'Moçambique', 'Portugal', 'São Tomé e Príncipe',
       'Timor-Leste'], dtype=object)

Por cada país obter os codigos com detalhes dos produtos

In [84]:
country_cmd_top5_codes = dict()
for country in countries:
    l2_codes = pco_top5[pco_top5.partnerDesc == country]['cmdCode'].unique()
    print(country,l2_codes)
    hs_details = []
    for l2_code in l2_codes:
        l2_sub_codes = list(hs_codes_df[hs_codes_df.hscode.str.startswith(l2_code)]['hscode'])
        hs_details = hs_details + l2_sub_codes
    # print(hs_details)
    country_cmd_top5_codes[country] = hs_details.copy()


Angola ['27' '25' '26' '74' '76']
Brazil ['26' '12' '27' '02' '47']
Cabo Verde ['74' '26' '62' '49' '99']
Guiné Equatorial ['27' '44' '90' '99' '84']
Guiné-Bissau ['99']
Moçambique ['26' '27' '44' '12' '74']
Portugal ['85' '87' '61' '84' '74']
São Tomé e Príncipe ['85' '84' '39' '90' '99']
Timor-Leste ['27' '09' '14' '85' '44']


TODO: pegar o detalhe HS de cada país a partir do top5

* isolar numa função o pedido e criação do DataFrame

In [86]:
country_cmd_top5_codes['Angola']

['27',
 '2701',
 '270111',
 '270112',
 '270119',
 '270120',
 '2702',
 '270210',
 '270220',
 '2703',
 '270300',
 '2704',
 '270400',
 '2705',
 '270500',
 '2706',
 '270600',
 '2707',
 '270710',
 '270720',
 '270730',
 '270740',
 '270750',
 '270791',
 '270799',
 '2708',
 '270810',
 '270820',
 '2709',
 '270900',
 '2710',
 '271012',
 '271019',
 '271020',
 '271091',
 '271099',
 '2711',
 '271111',
 '271112',
 '271113',
 '271114',
 '271119',
 '271121',
 '271129',
 '2712',
 '271210',
 '271220',
 '271290',
 '2713',
 '271311',
 '271312',
 '271320',
 '271390',
 '2714',
 '271410',
 '271490',
 '2715',
 '271500',
 '2716',
 '271600',
 '25',
 '2501',
 '250100',
 '2502',
 '250200',
 '2503',
 '250300',
 '2504',
 '250410',
 '250490',
 '2505',
 '250510',
 '250590',
 '2506',
 '250610',
 '250620',
 '2507',
 '250700',
 '2508',
 '250810',
 '250830',
 '250840',
 '250850',
 '250860',
 '250870',
 '2509',
 '250900',
 '2510',
 '251010',
 '251020',
 '2511',
 '251110',
 '251120',
 '2512',
 '251200',
 '2513',
 '251310',